In [2]:
train <- read.csv('kc_house_train_data.csv', header = TRUE)

In [3]:
test <- read.csv('kc_house_test_data.csv', header = TRUE)

In [4]:
sales <- rbind(train,test)

In [5]:
#3
get_matrix_data <- function(data_dframe, features, output){
    data_dframe$constant = 1
    features = c('constant',features)
    feature_dframe = data_dframe[,features]
    feature_matrix = data.matrix(feature_dframe)
    
    output_dframe = data_dframe[,output]
    output_matrix = data.matrix(output_dframe)
    return (list('feature_matrix'=feature_matrix,'output_matrix'=output_matrix))
}

In [6]:
example = get_matrix_data(train,'sqft_living','price')

In [7]:
example$feature_matrix[1,]
example$output_matrix[1]

constant sqft_living 
          1        1180

[1] 221900

In [8]:
#Predicting output given regression weights
my_weight = as.matrix(c(1,1))
my_feature = example$feature_matrix[1,]
predicted_value = my_feature %*% my_weight
print(predicted_value)

     [,1]
[1,] 1181


In [9]:
#predict output
predict_output <- function(feature_matrix, weights){
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = feature_matrix %*% weights
    return(predictions)
}

In [10]:
test_predictions = predict_output(example$feature_matrix, my_weight)
print(test_predictions[1])
print(test_predictions[2])


[1] 1181
[1] 2571


In [11]:
# Computing the Derivative
feature_derivative <- function(err,feat){
    derivatives = 2*(err * feat)
    return(sum(derivatives))
}

In [12]:
example = get_matrix_data(sales, 'sqft_living','price')
my_weight = as.matrix(c(0,0))
test_predictions = predict_output(example$feature_matrix, my_weight)
err = test_predictions - example$output_matrix

In [13]:
feat = example$feature_matrix[,1]

In [16]:
print(sum(-example$output_matrix)*2)

[1] -23345850016


In [18]:
#due to error 'Cannot allocate vector of size 3.5gb', lets check for the first 5
deriv = feature_derivative(err,feat)

In [19]:
print(deriv)

[1] -23345850016


In [20]:
#GRADIENT DECENT
regression_gradient_decent <- function(feature_matrix, output, initial_weight,step_size, tolerance){
    converged = FALSE
    weights = initial_weight
    while(!(converged)){
        predictions = predict_output(feature_matrix, weights)
        
        error = predictions - output
        gradient_sum_square = 0
        for(i in 1:length(weights)){
            derivative =  feature_derivative(error,feature_matrix[,i])
            
            gradient_sum_square = gradient_sum_square+(derivative**2)
            
            weights[i] =weights[i]-(step_size*derivative)
        }
        gradient_magnitude = sqrt(gradient_sum_square)
        if(gradient_magnitude < tolerance){
            converged = TRUE
        } 
    }
    return(weights)
}

In [21]:
#RUNNING GRADIENT DECENT
simple_feature <- 'sqft_living'
my_output <- 'price'
simple_matrix <- get_matrix_data(train,simple_feature,my_output)
initial_weight = c(-47000,1)
step_size=7e-12
tolerance = 2.5e7

In [22]:
test_weight = regression_gradient_decent(simple_matrix$feature_matrix, simple_matrix$output_matrix,initial_weight,step_size,tolerance)

In [23]:
test_weight

[1] -46999.8872    281.9121

In [29]:
test_matrix <- get_matrix_data(test,'sqft_living','price')
test_prediction <- predict_output(test_matrix$feature_matrix,test_weight)

In [30]:
test_residual <- test_prediction - test_matrix$output_matrix
test_RSS = sum(test_residual*test_residual)
test_RSS

[1] 2.754e+14

In [ ]:
#RUNNING MULTIPLE REGRESSION

In [31]:
feature_matrix_train <- c('sqft_living','sqft_living15')
my_output <- 'price'
matrix_test <- get_matrix_data(train,feature_matrix_test,my_output)

In [32]:
initial_weight <- as.matrix(c(-100000,1,1))
step_size <- 4e-12
tolerance <- 1e+9

In [33]:
weight_2 <- regression_gradient_decent(matrix_test$feature_matrix, matrix_test$output_matrix,initial_weight,step_size,tolerance)

In [34]:
weight_2

-99999.96885
245.07260
65.27953


In [35]:
feature_matrix_test <- c('sqft_living','sqft_living15')
my_output <- 'price'
matrix_test <- get_matrix_data(test,feature_matrix_test,my_output)

In [38]:
test_prediction_mr <- predict_output(matrix_test$feature_matrix,weight_2)
test_prediction_mr

366651.4
762662.4
386312.1
636989.7
269618.0
713122.6
1091530.1
598292.3
293002.1
672521.8
465322.3


In [39]:
matrix_test$output_matrix[0]

numeric(0)

In [40]:
test_residual <- test_prediction_mr - matrix_test$output_matrix
test_RSS = sum(test_residual*test_residual)
test_RSS

[1] 2.702634e+14